In [1]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Config:
    # Data configuration
    data_dir: str = '/mnt/d/datasets/mvtec'
    categories: List[str] = field(default_factory=lambda: ['bottle'])
    img_size: int = 256
    batch_size: int = 32
    valid_ratio: float = 0.2
    seed: int = 42

config = Config()
config

Config(data_dir='/mnt/d/datasets/mvtec', categories=['bottle'], img_size=256, batch_size=32, valid_ratio=0.2, seed=42)

In [ ]:
from data import MVTecDataset, get_transforms, get_dataloader, split_train_valid

# Get transforms
train_transform, test_transform = get_transforms(img_size=config.img_size)

# Create datasets
train_dataset = MVTecDataset(config.data_dir, config.categories, 'train', transform=train_transform)
valid_dataset = MVTecDataset(config.data_dir, config.categories, 'train', transform=test_transform)
test_dataset = MVTecDataset(config.data_dir, config.categories, 'test', transform=test_transform)

# Split train/valid
train_dataset, valid_dataset = split_train_valid(
    train_dataset, valid_dataset, 
    valid_ratio=config.valid_ratio, 
    seed=config.seed
)

# Create dataloaders
train_loader = get_dataloader(train_dataset, config.batch_size, 'train')
valid_loader = get_dataloader(valid_dataset, 16, 'valid')
test_loader = get_dataloader(test_dataset, 16, 'test')

print(f'Train size: {len(train_loader.dataset)}')
print(f'Valid size: {len(valid_loader.dataset)}')
print(f'Test  size: {len(test_loader.dataset)}')

Train size: 168
Valid size: 41
Test  size: 83


In [ ]:
# Dataloader()

In [3]:
import torch
from modeler import Modeler, get_model, get_loss_fn, get_metric
from trainer import Trainer, get_optimizer, get_scheduler, get_logger

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = get_model('vanilla_ae', 
    in_channels=3, out_channels=3, latent_dim=128, img_size=256).to(device)
loss_fn = get_loss_fn('combined')
metrics = {"ssim": get_metric('ssim'), 
           "psnr": get_metric('psnr')}
modeler = Modeler(model, loss_fn, metrics)

optimizer = get_optimizer(model, "adamw", lr=0.001, weight_decay=1e-5)
scheduler = get_scheduler(optimizer, "plateau")
logger = get_logger("./experments")
trainer = Trainer(modeler, optimizer, scheduler, logger)

history = trainer.fit(train_loader, num_epochs=10, valid_loader=None)

Train [10/10]: 100%|██████████| 5/5 [00:03<00:00,  1.40it/s, loss=0.292, ssim=0.511, psnr=10.235]

Training completed!


In [ ]:
model = get_model('vae', 
    in_channels=3, out_channels=3, latent_dim=128, img_size=256).to(device)
loss_fn = get_loss_fn('vae')
metrics = {"ssim": get_metric('ssim'), 
           "psnr": get_metric('psnr')}
modeler = Modeler(model, loss_fn, metrics)

optimizer = get_optimizer(model, "adamw", lr=0.001, weight_decay=1e-5)
scheduler = get_scheduler(optimizer, "plateau")
logger = get_logger("./experments")
trainer = Trainer(modeler, optimizer, scheduler, logger)

history = trainer.fit(train_loader, num_epochs=10, valid_loader=None)

Train [10/10]: 100%|██████████| 5/5 [00:03<00:00,  1.40it/s, loss=0.054, ssim=0.486, psnr=12.971]

Training completed!
